In [5]:
import mysql.connector
import pandas as pd

connection = mysql.connector.connect(
    host='localhost',          
    user='root',      
    password='incubytes@123',  
)

## Creating Staging/Intermediate Table

In [ ]:
create_staging_table = """
CREATE TABLE IF NOT EXISTS Staging_Customers (
    Customer_Name VARCHAR(255) NOT NULL,
    Customer_Id INT NOT NULL,
    Open_Date DATE NOT NULL,
    Last_Consulted_Date DATE,
    Vaccination_Id CHAR(5),
    Dr_Name CHAR(255),
    State CHAR(5),
    Country CHAR(5),
    Post_Code INT,
    DOB DATE,
    Is_Active CHAR(1),
    PRIMARY KEY (Customer_Id)
);
"""
cursor.execute(create_staging_table)

## Loading data into staging table from Source data csv file 

In [6]:
import sqlite3

# Step 1: Load the CSV file into a DataFrame
csv_file_path = 'Source_data.csv'  # replace with the path to your CSV file

df = pd.read_csv(csv_file_path)
df.head()

# Step 2: Connect to your database 
conn = sqlite3.connect('hospital.db') 
cursor = conn.cursor()
df.head()

# Insert the data from the DataFrame into the database
try:
    for index, row in df.iterrows():
        cursor.execute("""
        INSERT INTO Staging_Customers (Customer_Name, Customer_Id, Open_Date, Last_Consulted_Date, Vaccination_Id, Dr_Name, State, Country, DOB, Is_Active) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, tuple(row))
    # Commit the transaction
    conn.commit()
    print("Data inserted successfully.")
except Exception as e:
    print(f"Error occurred: {e}")
conn.commit()

cursor.execute("SELECT * FROM Staging_Customers;")
rows = cursor.fetchall()

print(rows)
# Commit the transaction


Error occurred: UNIQUE constraint failed: Staging_Customers.Customer_Id
[('Aayushi Sharma', 123456, 20101012, 20211013, 'MVD', 'Dewang', 'AP', 'IND', 456010, 19980331, 'A'), ('Akshita Patidar', 123457, 20101013, 20211014, 'MVD', 'Yash', 'HR', 'IND', 456119, 19981111, 'A'), ('Shakti Aggarwal', 123458, 20210923, 20210924, 'MVD', 'Emma', 'NSW', 'JP', 408223, 19990129, 'A'), ('Anjali Sharma', 123459, 20210916, 20210917, 'MVD', 'Vera', 'WB', 'IND', 4, 19970203, 'N'), ('Ishika Tanwani', 123460, 20210807, 20210808, 'MVD', 'Shraddha', 'LM', 'USA', 41021, 19970612, 'N'), ('Khushi Jain', 123461, 20101012, 20210613, 'MVD', 'Paritosh', 'LA', 'USA', 67456, 19980409, 'N'), ('Shruti Jain', 123462, 20101013, 20210614, 'MVD', 'Parikshit', 'HQ', 'JP', 4006, 19981010, 'A'), ('Gaurav Mishra', 123463, 20101014, 20210615, 'MVD', 'Nayan', 'TV', 'JP', 'AB 101', 19970203, 'N'), ('Vaishali Jain', 123464, 20101015, 20210616, 'MVD', 'Ankur', 'AP', 'IND', 40061, 19981111, 'A'), ('Rima Bar', 123465, 20101016, 20210

## Calculating Age and Days Since Last Consulted column and adding into Source data.

In [4]:
from datetime import datetime

df['DOB'] = pd.to_datetime(df['DOB'], format='%Y%m%d')
df['Last_Consulted_Date'] = pd.to_datetime(df['Last_Consulted_Date'], format='%Y%m%d')

df['Age'] = (datetime.now() - df['DOB']).dt.days // 365
df['Days_Since_Last_Consulted'] = (datetime.now() - df['Last_Consulted_Date']).dt.days


# Creating Common Table containing all the columns including age and days since last consulted column

In [9]:
import pandas as pd
import sqlite3
from datetime import datetime

# Step 1: Load the CSV file into a DataFrame
csv_file_path = 'Source_data.csv'  
df = pd.read_csv(csv_file_path)

# Convert date columns from string to datetime
df['DOB'] = pd.to_datetime(df['DOB'], format='%Y%m%d')
df['Last_Consulted_Date'] = pd.to_datetime(df['Last_Consulted_Date'], format='%Y%m%d')
df['Open_Date'] = pd.to_datetime(df['Open_Date'], format='%Y%m%d')

# Calculate Age and Days Since Last Consulted
df['Age'] = (datetime.now() - df['DOB']).dt.days // 365
df['Days_Since_Last_Consulted'] = (datetime.now() - df['Last_Consulted_Date']).dt.days

# Convert date columns to string format for SQLite
date_columns = ['Open_Date', 'Last_Consulted_Date', 'DOB']
for column in date_columns:
    df[column] = df[column].dt.strftime('%Y-%m-%d')  # Format as needed

print(df.head())

conn = sqlite3.connect('hospital.db')  # added .db extension for clarity
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS common_table (
    Customer_Name TEXT,
    Customer_Id INTEGER,
    Open_Date TEXT,
    Last_Consulted_Date TEXT,
    Vaccination_Id INTEGER,
    Dr_Name TEXT,
    State TEXT,
    Country TEXT,
    DOB TEXT,
    Is_Active INTEGER,
    Age INTEGER,
    Days_Since_Last_Consulted INTEGER
)
""")

for index, row in df.iterrows():
    cursor.execute("""
    INSERT INTO common_table (Customer_Name, Customer_Id, Open_Date, Last_Consulted_Date, Vaccination_Id, Dr_Name, State, Country, DOB, Is_Active, Age, Days_Since_Last_Consulted) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, tuple(row))

conn.commit()


     Customer_Name  Customer_Id   Open_Date Last_Consulted_Date  \
0   Aayushi Sharma       123456  2010-10-12          2021-10-13   
1  Akshita Patidar       123457  2010-10-13          2021-10-14   
2  Shakti Aggarwal       123458  2021-09-23          2021-09-24   
3    Anjali Sharma       123459  2021-09-16          2021-09-17   
4   Ishika Tanwani       123460  2021-08-07          2021-08-08   

  Vaccination_Id   Dr_Name State Country         DOB Is_Active  Age  \
0            MVD    Dewang    AP     IND  1998-03-31         A   26   
1            MVD      Yash    HR     IND  1998-11-11         A   25   
2            MVD      Emma   NSW      JP  1999-01-29         A   25   
3            MVD      Vera    WB     IND  1997-02-03         N   27   
4            MVD  Shraddha    LM     USA  1997-06-12         N   27   

   Days_Since_Last_Consulted  
0                       1108  
1                       1107  
2                       1127  
3                       1134  
4              

# Creating separate tables country wise for example Table_IND containing all the data of indian candidates

In [10]:
import pandas as pd
import sqlite3
from datetime import datetime


conn = sqlite3.connect('hospital.db')
cursor = conn.cursor()

df = pd.read_sql_query("SELECT * FROM common_table", conn)


df['DOB'] = pd.to_datetime(df['DOB'])
df['Last_Consulted_Date'] = pd.to_datetime(df['Last_Consulted_Date'])
df['Open_Date'] = pd.to_datetime(df['Open_Date'])

# Calculate Age and Days Since Last Consulted
df['Age'] = (datetime.now() - df['DOB']).dt.days // 365
df['Days_Since_Last_Consulted'] = (datetime.now() - df['Last_Consulted_Date']).dt.days

# Convert date columns to string format for SQLite
date_columns = ['Open_Date', 'Last_Consulted_Date', 'DOB']
for column in date_columns:
    df[column] = df[column].dt.strftime('%Y-%m-%d')

# Create country-wise tables and print their contents
unique_countries = df['Country'].unique()

for country in unique_countries:
    table_name = f"Table_{country.replace(' ', '_').replace('-', '_')}"  # Replace spaces and hyphens with underscores
    
    # Create the table if it doesn't exist
    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        Customer_Name TEXT,
        Customer_Id INTEGER,
        Open_Date TEXT,
        Last_Consulted_Date TEXT,
        Vaccination_Id INTEGER,
        Dr_Name TEXT,
        State TEXT,
        Country TEXT,
        DOB TEXT,
        Is_Active INTEGER,
        Age INTEGER,
        Days_Since_Last_Consulted INTEGER
    )
    """)
    
    # Insert data specific to the current country
    country_data = df[df['Country'] == country]
    for index, row in country_data.iterrows():
        cursor.execute(f"""
        INSERT INTO {table_name} (Customer_Name, Customer_Id, Open_Date, Last_Consulted_Date, Vaccination_Id, Dr_Name, State, Country, DOB, Is_Active, Age, Days_Since_Last_Consulted) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, tuple(row))

  
    conn.commit()

    # Print the contents of the created table
    print(f"\nContents of {table_name}:")
    country_df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    print(country_df)




Contents of Table_IND:
      Customer_Name  Customer_Id   Open_Date Last_Consulted_Date  \
0    Aayushi Sharma       123456  2010-10-12          2021-10-13   
1   Akshita Patidar       123457  2010-10-13          2021-10-14   
2     Anjali Sharma       123459  2021-09-16          2021-09-17   
3     Vaishali Jain       123464  2010-10-15          2021-06-16   
4     Manav Kampani       123467  2010-10-18          2021-06-19   
..              ...          ...         ...                 ...   
75   Aayushi Sharma       123456  2010-10-12          2021-10-13   
76  Akshita Patidar       123457  2010-10-13          2021-10-14   
77    Anjali Sharma       123459  2021-09-16          2021-09-17   
78    Vaishali Jain       123464  2010-10-15          2021-06-16   
79    Manav Kampani       123467  2010-10-18          2021-06-19   

   Vaccination_Id Dr_Name State Country         DOB Is_Active  Age  \
0             MVD  Dewang    AP     IND  1998-03-31         A   26   
1             MVD  